In [85]:
import numpy as np
import pandas as pd
import h5py
from sklearn.metrics import classification_report, confusion_matrix

In [70]:
#Se importan los datasets, estos fueron descargados de Kaggle
mnist = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
mnist_test = pd.read_csv('fashionmnist/fashion-mnist_test.csv')

In [60]:
#Se asignan las variables que contienen los datos de las observaciones
x_train = mnist.drop('label', axis=1)
x_test = mnist_test.drop('label', axis=1)

In [72]:
#Se asignan las variables de las labels
y_train = np.int32(mnist['label'])
y_test = mnist_test['label']

In [67]:
#Se normalizan los datos de las observaciones
x_train = np.float32(x_train) / 255
x_test = np.float32(x_test) / 255

In [74]:
#Se realiza one-hot-encoding para tener varios arreglos con los labels
labels = 10
examples = y_train.shape[0]

y = y_train.reshape(1, examples)

y_new = np.eye(labels)[y.astype('int32')]
y_new = y_new.T.reshape(labels,examples)

In [77]:
#Se divide en test y train, 45000 obervaciones de entrenamiento, 15000 de testing
m = 45000
m_test = x_train.shape[0] - m

x, x_test = x_train[:m].T, x_train[m:].T
y, y_test = y_new[:, :m], y_new[:, m:]

In [79]:
#Sigmoide
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

#Costo
def compute_multiclass_loss(Y, Y_hat):

    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum

    return L

In [80]:
"""
Proceso de la red neural
"""
n_x = x.shape[0]
n_h = 64
learning_rate = 1

#INICIALIZACION DE PESOS Y BIAS
W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(labels, n_h)
b2 = np.zeros((labels, 1))

X = x
Y = y

for i in range(2000):
#FORWARD PROPAGATION STARTS
    Z1 = np.matmul(W1,X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2,A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    cost = compute_multiclass_loss(Y, A2)

    dZ2 = A2-Y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)
    
#BACKPROPAGATION STARTS
    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m) * np.matmul(dZ1, X.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)
    
#ACTUALIZACION DE PESOS Y BIAS
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
#IMPRIMIR COSTOS CADA 100 ITERACIONES (EPOCHS)
    if (i % 100 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)

Epoch 0 cost:  6.808279258142297
Epoch 100 cost:  2.317844216531321
Epoch 200 cost:  2.278810130795445
Epoch 300 cost:  2.247532203618345
Epoch 400 cost:  2.2228176320278448
Epoch 500 cost:  2.1977953151483036
Epoch 600 cost:  2.1770374234475502
Epoch 700 cost:  2.1472515848373437
Epoch 800 cost:  2.1292521913430917
Epoch 900 cost:  2.0926383756826286
Epoch 1000 cost:  2.059953208871929
Epoch 1100 cost:  2.0280553232187386
Epoch 1200 cost:  1.9959603496394127
Epoch 1300 cost:  1.963857022364321
Epoch 1400 cost:  1.932292801243872
Epoch 1500 cost:  1.9025229925007943
Epoch 1600 cost:  2.0123629897193704
Epoch 1700 cost:  1.8771715201075299
Epoch 1800 cost:  1.8183742183886673
Epoch 1900 cost:  1.8818745782949422
Final cost: 1.784041493404962


In [86]:
#ULTIMA ACTUALIZACION DE LOS PESOS
Z1 = np.matmul(W1, x_test) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = sigmoid(Z2)

predictions = (A2>.5)[0,:]
labels = (y_test == 1)[0,:]
#OBTENER UN REPORTE A TRAVES DE LA LIBRERIA DE SKLEARN, CLASSIFICATION REPORT
print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

[[12663   640]
 [  819   878]]
              precision    recall  f1-score   support

       False       0.94      0.95      0.95     13303
        True       0.58      0.52      0.55      1697

    accuracy                           0.90     15000
   macro avg       0.76      0.73      0.75     15000
weighted avg       0.90      0.90      0.90     15000

